In [ ]:
# @title IMPORTAÇÃO DAS BIBLIOTECAS
import psycopg2 as pg

In [ ]:
# @title CONEXÃO COM BANCO DE DADOS
USER = "postgres"
SENHA = "ZAQwsx123!@#"
HOST = "localhost"
DATABASE = "datatran"

try:
    conn = pg.connect(
            host = HOST,
            database = DATABASE,
            user = USER,
            password = SENHA)
except Exception as e:
    print("Erro na conexão.", type(e))